In [178]:
## Basic Packages
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re
from pprint import pprint

## Import NLTK packages
from nltk.corpus import stopwords
from nltk import RegexpTokenizer
from nltk import wordpunct_tokenize
from nltk.stem.porter import PorterStemmer
from nltk import pos_tag, word_tokenize
from sklearn.feature_extraction.text import TfidfTransformer

## Import Spacy
import spacy

## Import Gensim packages
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, TfidfModel

## Plotting Tools
import pyLDAvis
import pyLDAvis.gensim

## Notebook display settings
pd.options.display.max_columns = 999

## Reading in data

In [179]:
lyrics_df = pd.read_csv('data//lyric_data.csv', encoding = 'utf8')

FileNotFoundError: File b'data//lyric_data.csv' does not exist

In [180]:
beatles_lyrics = lyrics_df[lyrics_df.artist == 'The Beatles']

NameError: name 'lyrics_df' is not defined

In [ ]:
beatles_lyrics.drop(columns = ['link'], inplace = True)

In [ ]:
beatles_lyrics.reset_index(drop = True, inplace = True)

In [ ]:
beatles_lyrics.rename(columns = {'text' : 'lyrics'}, inplace = True)

In [ ]:
beatles_lyrics.to_csv('data//beatles_lyrics.csv')

## Text cleanup

In [2]:
beatles_lyrics = pd.read_csv('data//beatles_lyrics.csv')

In [3]:
## Creating a list of lyrics

stripped_lyrics = beatles_lyrics.lyrics.values.tolist()
stripped_lyrics = [x.lower() for x in stripped_lyrics]

In [4]:
## Cleaning up the text - removing new lines, extra spaces, random characters

stripped_lyrics = [re.sub('\n', '', sent) for sent in stripped_lyrics]
stripped_lyrics = [re.sub('\s{2,}', ' ', sent) for sent in stripped_lyrics]
stripped_lyrics = [re.sub("\\\'", '', sent) for sent in stripped_lyrics]

In [5]:
len(stripped_lyrics)

178

In [6]:
## Importing a stop word list and including a few extra words

stop_words = stopwords.words('english')
stop_words.extend(['hey', 'nah', '[chorus]', 'la', 'na', 'ta', 'll'])

In [126]:
## Tokenizing the lyrics in each song using RegexpTokenizer and removing stop words

tokenizer = RegexpTokenizer(r'\w+')
porter = PorterStemmer()

token_list = []

for i in range(len(stripped_lyrics)):   
    tokenized_list = []
    #tokenized_list = tokenizer.tokenize(stripped_lyrics[i])
    tokenized_list = word_tokenize(stripped_lyrics[i])
    tokenized_list = [i for i in tokenized_list if i not in stop_words]
    #tokenized_list = [pos_tag(word_tokenize(tokens), tagset='universal') for tokens in tokenized_list]
    #tokenized_list = [porter.stem(tokens) for tokens in tokenized_list]
    token_list.append(tokenized_list)

In [127]:
longer_token_list = []

for i in token_list:
    iterate_list = []
    for j in i:
        if ((len(j) > 3) == True):
            iterate_list.append(j)
    longer_token_list.append(iterate_list)

In [128]:
token_list = longer_token_list

In [104]:
nlp = spacy.load('en', disable=['parser', 'ner'])

/Users/justin/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/Users/justin/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
/Users/justin/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/Users/justin/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])


In [116]:
def lemmatization(texts, allowed_postags=['NOUN', 'VERB', 'ADJ', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [117]:
token_list = lemmatization(token_list, allowed_postags=['NOUN', 'VERB', 'ADJ', 'ADV'])

In [118]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(token_list, min_count=5, threshold=70) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[token_list], threshold=70)  

# Faster way to get a sentence labeled as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/Users/justin/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [129]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [130]:
data_words_bigrams = make_bigrams(token_list)
data_words_trigrams = make_trigrams(token_list)

In [131]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_trigrams)

# Create Corpus
texts = data_words_trigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 3), (6, 4), (7, 3), (8, 2), (9, 3), (10, 3), (11, 1), (12, 3), (13, 3), (14, 6), (15, 1), (16, 1), (17, 3), (18, 1), (19, 6), (20, 2), (21, 1), (22, 1), (23, 5), (24, 3), (25, 1), (26, 2), (27, 3), (28, 2), (29, 1), (30, 3), (31, 1), (32, 1), (33, 7), (34, 2), (35, 3), (36, 3), (37, 3), (38, 4), (39, 5), (40, 3), (41, 5), (42, 1), (43, 7), (44, 1), (45, 1), (46, 1), (47, 3), (48, 3), (49, 9), (50, 3), (51, 3)]]


In [132]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5,
                                           random_state=42,
                                           update_every=10,
                                           chunksize=50,
                                           passes=4,
                                           alpha='auto',
                                           per_word_topics=True)

In [173]:
tfidf_model = gensim.models.TfidfModel(corpus, id2word = id2word)

lda_model = gensim.models.ldamodel.LdaModel(tfidf_model[corpus], id2word = id2word, num_topics = 13)

In [174]:
# Print the Keyword in the 5 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.008*"yeah" + 0.004*"sunshine" + 0.004*"hand" + 0.004*"wont" + '
  '0.004*"commonwealth" + 0.004*"youve" + 0.004*"loving" + 0.003*"coming" + '
  '0.003*"leave" + 0.003*"blackbird"'),
 (1,
  '0.005*"hello" + 0.004*"help" + 0.004*"lucille" + 0.004*"well" + '
  '0.003*"cross_town" + 0.003*"roll_mystery_tour" + 0.003*"never" + '
  '0.003*"waited" + 0.003*"bulldog" + 0.003*"dreamers"'),
 (2,
  '0.005*"christmas" + 0.005*"dear_prudence" + 0.005*"to-night" + '
  '0.004*"tonight" + 0.004*"lucy" + 0.004*"diamonds" + 0.003*"black" + '
  '0.003*"hands" + 0.003*"keep" + 0.003*"molly"'),
 (3,
  '0.007*"better" + 0.005*"robert" + 0.005*"girl" + 0.004*"back" + '
  '0.004*"together" + 0.003*"week" + 0.003*"thing" + 0.003*"cheat" + '
  '0.003*"jude" + 0.003*"belonged"'),
 (4,
  '0.008*"want" + 0.005*"money" + 0.004*"thats" + 0.004*"komm" + '
  '0.004*"mister_moonlight" + 0.004*"guilty" + 0.004*"misery" + '
  '0.004*"evrything" + 0.003*"love" + 0.003*"needed"'),
 (5,
  '0.005*"rich" + 0.004*"bo

In [175]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) 

coherence_model_lda = CoherenceModel(model = lda_model, texts = texts, dictionary = id2word, coherence = 'c_v')
coherence_lda = coherence_model_lda.get_coherence()

print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.94584329035625

Coherence Score:  0.5649375726137557


In [176]:
## Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

In [177]:
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
6      14.403443        1       1 -0.041007 -0.032080
0      12.408198        1       2 -0.016000  0.028289
12     11.303028        1       3 -0.011661  0.025612
9       9.946286        1       4  0.006024  0.005648
4       9.911359        1       5  0.027708 -0.014848
8       6.915511        1       6 -0.002599 -0.000973
3       6.910463        1       7  0.001799  0.004962
1       5.960816        1       8  0.003492 -0.006077
10      5.902705        1       9  0.015751 -0.010476
7       5.299176        1      10  0.008878 -0.003985
5       4.601443        1      11  0.005674  0.004154
2       3.742270        1      12  0.001577 -0.000146
11      2.695306        1      13  0.000364 -0.000081, topic_info=     Category       Freq           Term      Total  loglift  logprob
term                                                                
152   Default  17.000000         better  17.000000  30.0000  30.0000
113   Default  27.000000           want  27.000000  29.0000  29.0000
116   Default  33.000000           yeah  33.000000  28.0000  28.0000
516   Default  11.000000          hello  11.000000  27.0000  27.0000
160   Default  15.000000           shes  15.000000  26.0000  26.0000
56    Default  17.000000           call  17.000000  25.0000  25.0000
336   Default  11.000000           john  11.000000  24.0000  24.0000
929   Default   6.000000      christmas   6.000000  23.0000  23.0000
327   Default   9.000000          brian   9.000000  22.0000  22.0000
532   Default  11.000000         needed  11.000000  21.0000  21.0000
1123  Default   6.000000  dear_prudence   6.000000  20.0000  20.0000
251   Default  10.000000       anything  10.000000  19.0000  19.0000
908   Default   7.000000   carry_weight   7.000000  18.0000  18.0000
12    Default  14.000000          dance  14.000000  17.0000  17.0000
1392  Default   6.000000       to-night   6.000000  16.0000  16.0000
22    Default  11.000000           help  11.000000  15.0000  15.0000
403   Default   8.000000         robert   8.000000  14.0000  14.0000
155   Default  24.000000           girl  24.000000  13.0000  13.0000
213   Default   6.000000           rich   6.000000  12.0000  12.0000
40    Default  10.000000           sing  10.000000  11.0000  11.0000
182   Default  10.000000        tonight  10.000000  10.0000  10.0000
329   Default  10.000000          comes  10.000000   9.0000   9.0000
1233  Default   7.000000         johnny   7.000000   8.0000   8.0000
1141  Default   6.000000         bother   6.000000   7.0000   7.0000
218   Default  11.000000           glad  11.000000   6.0000   6.0000
566   Default  10.000000          child  10.000000   5.0000   5.0000
1330  Default   5.000000     hallelujah   5.000000   4.0000   4.0000
512   Default  13.000000        goodbye  13.000000   3.0000   3.0000
738   Default   7.000000      must_pass   7.000000   2.0000   2.0000
1119  Default   6.000000        tripper   6.000000   1.0000   1.0000
...       ...        ...            ...        ...      ...      ...
410   Topic13   0.691910          cried   6.090171   1.4387  -6.1939
40    Topic13   1.144032           sing  10.071555   1.4385  -5.6910
57    Topic13   0.582620          calls   5.256924   1.4139  -6.3658
420   Topic13   0.641798          needs   6.050047   1.3701  -6.2690
612   Topic13   0.665612          wants   6.359531   1.3567  -6.2326
684   Topic13   0.671843         rodney   6.573331   1.3329  -6.2233
218   Topic13   1.085615           glad  11.026860   1.2955  -5.7434
1525  Topic13   0.472478       surprise   4.814538   1.2923  -6.5753
110   Topic13   0.535918          phone   5.474280   1.2898  -6.4493
688   Topic13   0.445785          third   4.787842   1.2397  -6.6335
677   Topic13   0.445785           form   4.787843   1.2397  -6.6335
672   Topic13   0.445785       brighten   4.787842   1.2397  -6.6335
686   Topic13   0.445785       swinging   

## TF-IDF on lyrics

In [20]:
#print(token_list)

In [111]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_1 = CountVectorizer()
matrix = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False).fit_transform(token_list)
matrix.toarray()
#matrix = vectorizer.fit_transform(token_list)
#print(stemmed_words)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int64)

In [125]:
from sklearn.feature_extraction.text import TfidfTransformer

no_features = 500

transformer = TfidfTransformer()
tfidf_transform = transformer.fit_transform(matrix)

In [126]:
tfidf_transform

<178x1724 sparse matrix of type '<class 'numpy.float64'>'
	with 6409 stored elements in Compressed Sparse Row format>

In [124]:
tfidf_vectorizer = TfidfVectorizer(max_df=.85, min_df=2, max_features=no_features)
tfidf = tfidf_vectorizer.fit_transform(tfidf_transform)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

AttributeError: lower not found

In [116]:
no_topics = 5

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=0.05, l1_ratio=.5, init='nndsvd').fit(tfidf)

In [131]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        for k in topic.argsort():
            print("Word: ",feature_names[k], "  Score: ", topic[k]/ sum(topic))

no_top_words = 10

display_topics(nmf, tfidf_feature_names, no_top_words)

Topic 0:
Word:  act   Score:  0.0


IndexError: list index out of range

In [100]:
no_features = 500

documents = beatles_lyrics.lyrics.values.tolist()

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=.85, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=.85, min_df=2, max_features=no_features, stop_words= stop_words)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [101]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 5

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=0.05, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=20,random_state=0).fit(tf)

In [102]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
don ll ve want time got know come let just
Topic 1:
love know need ll girl true whoa loved make heart
Topic 2:
yeah oh got john good said brian feeling come end
Topic 3:
baby man want old glad know trying clarabella gonna sister
Topic 4:
hey talk boys better bye ah bulldog ya ha make
Topic 0:
love yeah want let baby oh know come well like
Topic 1:
back baby cry fun get got honey sky lucy diamonds
Topic 2:
go yeah long want oh hand day know good way
Topic 3:
know bye say please time never love hello help oh
Topic 4:
oh girl john mm love di falling yes come right
